# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [72]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [73]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [74]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms'] = sales['bedrooms'].astype(float)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 

sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [75]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [76]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 0.341260     | 6899658.604835     | 386801.941137                   |

| 10        | 11       | 0.000002  | 0.452672     | 6839835.698640     | 362470.635108                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [81]:
# model_all
rows = len(all_features)
model_all.coefficients.print_rows(num_rows=100)


+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 278815.4192069245  |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  2.25 |        0.0         |  None  |
|    bathrooms     |   3   |        0.0         |  None  |
|    bathrooms     |   2   |        0.0         |  None  |
|    bathrooms     |  4.5  |        0.0         |  None  |
|    bathrooms     |  1.5  |        0.0         |  None  |
|    bathrooms     |  2.5  |        0.0         |  None  |
|    bathrooms     |  1.75 |        0.0         |  None  |
|    bathrooms     |  2.75 |        0.0         |  None  |
|    bathrooms     |  3.25 |        0.0         |  None  |
|    bathrooms     |   4   |        0.0         |  None  |
|    bathrooms     |  3.5  |        0.0         |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [82]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [83]:
import numpy as np
np_logspace = np.logspace(1,7,num=13)
num_penality = len(np_logspace)
RSS = np.empty(num_penality)
for i in range(num_penality):
    model = turicreate.linear_regression.create(training,target='price',features=all_features,l1_penalty=np_logspace[i],l2_penalty=0,validation_set=None,verbose=False)
    prediction = model.predict(validation)
    errors = prediction - validation['price']
    RSS[i] = np.dot(errors,errors)
print (RSS)

[5.96734420e+14 5.96734418e+14 5.96734413e+14 5.96734397e+14
 5.96734346e+14 5.96734184e+14 5.96733673e+14 5.96732057e+14
 5.96726948e+14 5.96710811e+14 5.96659975e+14 5.96500894e+14
 5.96017025e+14]


In [47]:
rss_min = RSS.min()
#print rss_min
for i in range(num_penality):
    #print RSS[i]
    #print np_logspace[i]
    if RSS[i]==rss_min:
        print (np_logspace[i])


10000000.0


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [86]:
model_l1 = turicreate.linear_regression.create(training,target='price',features=all_features,l1_penalty=10,l2_penalty=0,validation_set=None,verbose=False)
model_l1.coefficients.print_rows(num_rows=100)
model_l1.coefficients['value'].nnz()

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |  26997.603306532015  |  None  |
|     bedrooms     |  None |  8982.613590199953   |  None  |
| bedrooms_square  |  None |  888.7541345886539   |  None  |
|    bathrooms     |  4.5  |  350189.0581618711   |  None  |
|    bathrooms     |   1   |  -21810.24803881651  |  None  |
|    bathrooms     |  1.75 | -10413.242109185741  |  None  |
|    bathrooms     |  2.5  |  10226.116398392413  |  None  |
|    bathrooms     |  1.5  | -23605.189450993414  |  None  |
|    bathrooms     |  2.25 |  6911.793946201739   |  None  |
|    bathrooms     |   2   | -19423.377941221694  |  None  |
|    bathrooms     |  2.75 |  46101.34709294344   |  None  |
|    bathrooms     |  3.25 |  196526.29085178726  |  None  |
|    bathrooms     |  3.5  |  168637.0018287444   |  None  |
|    bathrooms     |  0.

44

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [51]:
# max_nonzeros = 7

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [52]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [56]:
np_logspace = np.logspace(8, 10, num=20)
num_penality = len(np_logspace)
RSS = np.empty(num_penality)
for i in range(num_penality):
    model = turicreate.linear_regression.create(training,target='price',features=all_features,l1_penalty=np_logspace[i],l2_penalty=0,validation_set=None,verbose=False)
    prediction = model.predict(validation)
    errors = prediction - validation['price']
    RSS[i] = np.dot(errors,errors)
    nnz = model.coefficients['value'].nnz()
    print (str(i) + ': '+ str(nnz))
print (RSS)

# 0: 18

0: 41
1: 38
2: 39
3: 38
4: 34
5: 32
6: 30
7: 27
8: 25
9: 20
10: 16
11: 14
12: 13
13: 12
14: 10
15: 6
16: 5
17: 1
18: 1
19: 1
[5.92783044e+14 5.92874597e+14 5.92851165e+14 5.94382692e+14
 5.98553343e+14 6.05469738e+14 6.17769708e+14 6.35012449e+14
 6.58572454e+14 6.87271855e+14 7.08975361e+14 7.44168048e+14
 8.02545133e+14 8.75802386e+14 9.88355296e+14 1.09699769e+15
 1.22064382e+15 1.27696276e+15 1.23093046e+15 1.22276795e+15]


In [57]:
print(np_logspace)

[1.00000000e+08 1.27427499e+08 1.62377674e+08 2.06913808e+08
 2.63665090e+08 3.35981829e+08 4.28133240e+08 5.45559478e+08
 6.95192796e+08 8.85866790e+08 1.12883789e+09 1.43844989e+09
 1.83298071e+09 2.33572147e+09 2.97635144e+09 3.79269019e+09
 4.83293024e+09 6.15848211e+09 7.84759970e+09 1.00000000e+10]


Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [59]:
l1_penalty_min = np_logspace[14]
l1_penalty_max = np_logspace[15]
print(l1_penalty_min)
print(l1_penalty_max)

2976351441.6313133
3792690190.7322536


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [60]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [63]:
num_l1 = len(l1_penalty_values)
RSS = np.empty(num_l1)
for i in range(num_l1):
    model = turicreate.linear_regression.create(training,target='price',features=all_features,l1_penalty=l1_penalty_values[i],l2_penalty=0,validation_set=None,verbose=False)
    prediction = model.predict(validation)
    errors = prediction - validation['price']
    RSS[i] = np.dot(errors,errors)
    nnz = model.coefficients['value'].nnz()
    print (str(i) + ': '+ str(nnz))
print (RSS)

0: 10
1: 9
2: 9
3: 9
4: 9
5: 9
6: 9
7: 9
8: 9
9: 8
10: 8
11: 7
12: 7
13: 7
14: 7
15: 7
16: 7
17: 7
18: 6
19: 6
[9.88355296e+14 9.94927574e+14 1.00109778e+15 1.00733087e+15
 1.01362408e+15 1.01997602e+15 1.02638638e+15 1.03289331e+15
 1.04036175e+15 1.04606602e+15 1.05143204e+15 1.05698874e+15
 1.06148602e+15 1.06627318e+15 1.07104803e+15 1.07626670e+15
 1.08216224e+15 1.08811173e+15 1.09286898e+15 1.09699769e+15]


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [66]:
rss_min = RSS.min()
print (rss_min)
for i in range(num_l1):
    #print RSS[i]
    #print np_logspace[i]
    if RSS[i]==rss_min:
        print (i)
        print (l1_penalty_values[i])

988355295831963.1
0
2976351441.6313133


In [70]:
model = turicreate.linear_regression.create(training,target='price',features=all_features,l1_penalty=l1_penalty_values[11],l2_penalty=0,validation_set=None,verbose=False)
model.coefficients.print_rows(num_rows=100)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 226472.83519563335 |  None  |
|     bedrooms     |  None | 2008.9791408011513 |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  4.5  |        0.0         |  None  |
|    bathrooms     |   1   |        0.0         |  None  |
|    bathrooms     |  1.75 |        0.0         |  None  |
|    bathrooms     |  2.5  |        0.0         |  None  |
|    bathrooms     |  1.5  |        0.0         |  None  |
|    bathrooms     |  2.25 |        0.0         |  None  |
|    bathrooms     |   2   |        0.0         |  None  |
|    bathrooms     |  2.75 |        0.0         |  None  |
|    bathrooms     |  3.25 |        0.0         |  None  |
|    bathrooms     |  3.5  |        0.0         |  None  |
|    bathrooms     |  0.75 |        0.0         |  None 

In [68]:
model

Class                          : LinearRegression

Schema
------
Number of coefficients         : 44
Number of examples             : 9761
Number of feature columns      : 17
Number of unpacked features    : 17

Hyperparameters
---------------
L1 penalty                     : 3448968612.1634
L2 penalty                     : 0.0

Training Summary
----------------
Solver                         : fista
Solver iterations              : 10
Solver status                  : Completed (Iteration limit reached).
Training time (sec)            : 0.2249

Settings
--------
Residual sum of squares        : 1187319267124410.8
Training RMSE                  : 353126.814

Highest Positive Coefficients
-----------------------------
(intercept)                    : 226472.8352
grade                          : 3511.7337
floors                         : 2792.8387
bedrooms                       : 2008.9791
sqft_living_sqrt               : 799.3534

Lowest Negative Coefficients
----------------------------

In [69]:
model.coefficients


name,index,value,stderr
(intercept),None,226472.83519563335,None
bedrooms,None,2008.9791408011513,None
bedrooms_square,None,0.0,None
bathrooms,4.5,0.0,None
bathrooms,1,0.0,None
bathrooms,1.75,0.0,None
bathrooms,2.5,0.0,None
bathrooms,1.5,0.0,None
bathrooms,2.25,0.0,None
bathrooms,2,0.0,None
